This program predict if the price a product is sold for is higher than the average price for the entire product category or not

the prediction is done using logistic regression and KNN

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
from sklearn.preprocessing import StandardScaler

In [4]:
df = pd.read_csv('e-shop clothing 2008.csv', sep=';', )

df

In [5]:
df.shape

In [6]:
dfNew = df.loc[(df["country"] == 43) | (df["country"] == 44) | (df["country"] == 45) | (df["country"] == 46) | (
        df["country"] == 46) | (df["country"] == 47)]
print(dfNew.shape)
df[df['country'] == '43-biz']

In [7]:
df.columns

In [8]:
# change some header name
df = df.rename(columns={
    "session ID": "session_ID",
    "page 1 (main category)": "category",
    "page 2 (clothing model)": "page_2",
    "model photography": "model_photography",
    "price 2": "is_higher_than_avg"
})

In [9]:
df = df.drop(["year", "page_2"], axis=1)

In [10]:
df.isna().sum()

In [11]:
df['is_higher_than_avg'] = (df['is_higher_than_avg'] == 1).astype(int)

In [12]:
df['model_photography'] = (df['model_photography'] == 1).astype(int)

In [13]:
df.head()

In [14]:
df['page'].unique()

In [15]:
# Create a figure
fig = plt.gcf()

# Change seaborn plot size
fig.set_size_inches(10, 6)


In [16]:
# remove the first one
df.corr()["is_higher_than_avg"].sort_values(ascending=False)[1:]
# df.corr()["price"].sort_values(ascending=False)[1:]

In [17]:
#Correlation Graph

plt.figure(figsize=(18, 8))

sn.heatmap(df.corr(), annot=True, cmap="Greens")

In [18]:
df.dtypes

In [19]:
# place is_higher_than_avg at the end of the data frame
is_higher_than_avg = df["is_higher_than_avg"]

df.drop("is_higher_than_avg", axis=1, inplace=True)

df["is_higher_than_avg"] = is_higher_than_avg

is_higher_than_avg

In [20]:
columns_name = df.columns
target = "price"

for label in columns_name:
    if label == target:
        continue

In [21]:
df['is_higher_than_avg'].plot.hist()

In [22]:
df_len = len(df)

train, test = np.split(df.sample(frac=1), [int(0.8 * df_len)])

print(df_len, len(train), len(test))

In [23]:
# , show_standardize = False
def scale_dataset(data_frame):
    X = data_frame[data_frame.columns[:-1]].values  # these are the columns needed for the prediction
    y = data_frame[data_frame.columns[-1]].values  # this is the predicted column

    # if show_standardize:
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    data = np.hstack((X, np.reshape(y, (-1, 1))))

    return data, X, y

In [24]:
train, X_train, y_train = scale_dataset(train)
test, X_test, y_test = scale_dataset(test)

In [25]:
X_train.shape

In [26]:
X_test.shape

In [27]:
# import necessary libraries for KNN model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [28]:
# Train
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

# predict
y_pred = knn_model.predict(X_test)
print(classification_report(y_test, y_pred))

**Logistic Regression**

In [29]:
from sklearn.linear_model import LogisticRegression

In [30]:
# train

lg_model = LogisticRegression()
lg_model = lg_model.fit(X_train, y_train)

# predict

y_pred = lg_model.predict(X_test)
print(classification_report(y_test, y_pred))

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics


In [32]:
# train
lr_model = LinearRegression()
lr_model = lr_model.fit(X_train, y_train)

# predict
y_pred = lr_model.predict(X_train)
accuracy = r2_score(y_train, y_pred)

print(f'Accuracy of trained dataset is {accuracy}')

y_pred = lr_model.predict(X_test)
accuracy = r2_score(y_test, y_pred)

print(f'Accuracy of test dataset is {accuracy}')

In [33]:
# Evaluate Linear Regression with
mae_lr = metrics.mean_absolute_error(y_test, y_pred)
mse_lr = metrics.mean_squared_error(y_test, y_pred)
rmse_lr = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

print(f'The mean absolute error is {mae_lr}.')
print(f'The mean squared error is {mse_lr}.')
print(f'The root mean squared error is {rmse_lr}.')

In [34]:
rsqLinReg = (lr_model.score(X_train, y_train)).round(3)
print(f"coefficient of determination: {rsqLinReg}")